In [77]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [ ]:
import numpy as np
seasons = np.arange(9,21)

urls = []
for season in seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelor_(season_{})'.format(season)
    urls.append(season_url)

In [62]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [141]:
import re

def dataCollector(soup):
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key with season number value pairs
    tbl_d['season'] = [num] * len(dic['age'])
    
    # clean up the names 
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\d+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # return the dict
    return tbl_d

In [155]:
dic = dataCollector('https://en.wikipedia.org/wiki/The_Bachelor_(season_18)')

In [156]:
pd.DataFrame(dic)

,age,elimination,hometown,name,occupation,season
0,26,Winner,"Kearney, Missouri",Nikki Ferrell,Pediatric Nurse,18
1,32,Runner-up,"Sacramento, California",Clare Crawley,Hairstylist,18
2,26,Quit in week 9,"Buckhead, Georgia",Andi Dorfman,Assistant District Attorney,18
3,32,Eliminated in week 8,"Martha's Vineyard, Massachusetts[a]",Renee Oteri,Real Estate Agent,18
4,24,Eliminated in week 7,"Lexington, Ohio",Chelsie Webster,Science Educator,18
5,29,Withdrew in week 7,"Ottawa, Ontario",Sharleen Joynt,Opera Singer,18
6,29,Eliminated in week 6,"Iowa City, Iowa",Kat Hurd,Medical Sales Representative/NBA Dancer,18
7,22[1],Eliminated in week 6,"Shelby Charter Township, Michigan",Cassandra Ferguson,Former NBA Dancer,18
8,26,Eliminated in week 5,"Orland Park, Illinois",Alli Restko,Nanny,18
9,25,Eliminated in week 5,"Litchfield, Illinois",Danielle Ronco,Psychiatric Nurse,18


,age,elimination,hometown,name,occupation,season
0,25,Winner,"Dallas, Texas",Melissa Rycroft,Sales Representative,13
1,24,Runner-up,"Brookfield, Wisconsin",Molly Malaney,Department Store Buyer,13
2,29,Eliminated in week 7,"Peace River, Alberta",Jillian Harris,Interior Designer,13
3,24,Eliminated in week 6,"Carlsbad, California",Naomi Rose Crespo,Flight Attendant,13
4,34,Eliminated in week 5,"Huntsville, Alabama",Stephanie Hogan,Single Mother & Medical Marketing Rep.,13
5,27,Eliminated in week 4,"Marlboro, New Jersey",Lauren Wanger,Teacher,13
6,25,Eliminated in week 4,"Sewickley, Pennsylvania",Megan Parris,Single Mother and Lacrosse Coach,13
7,29,Eliminated in week 4,"Marshall, Missouri",Shannon Bair,Dental Hygienist,13
8,29,Eliminated in week 4,"Blue Island, Illinois","Nicole ""Nikki"" Kaapke",Administrative Assistant,13
9,25,Eliminated in week 3,"Monroe, Connecticut",Erica DeSimone,Account Executive,13
